In [1]:
from qiskit.optimization import QuadraticProgram
from docplex.mp.advmodel import AdvModel
from docplex.mp.model_reader import ModelReader

from qiskit.optimization.algorithms import MinimumEigenOptimizer

from dwave.plugins.qiskit import DWaveMinimumEigensolver
from dwave.system import DWaveCliqueSampler, LeapHybridSampler

import numpy as np

# Test integer instances from Burkardt
## taken from https://people.math.sc.edu/Burkardt/datasets/datasets.html

In [2]:
def createSolver(sampler, num_reads = None ):

    if num_reads is None: # use sampler default
        dwave_solver = DWaveMinimumEigensolver(sampler =  sampler)
    else:
        dwave_solver = DWaveMinimumEigensolver(sampler =  sampler, num_reads = num_reads)
    return MinimumEigenOptimizer(dwave_solver)

In [3]:
def createModel(filename,model_name):
    model = ModelReader.read(filename=filename,model_name = model_name, model_class=AdvModel)
    qp = QuadraticProgram()
    qp.from_docplex(model)
    qp.model = model
    return qp

## at first we create our models and check the problems classically

In [4]:
import os
path = 'BURKARDT_DATA/INT/'

_, _, filenames = next(os.walk(path))
qps = {}
for file in filenames:
    name,_ = os.path.splitext(file)
    qp = createModel(path+file,name)
    qps[name] = qp    

In [5]:
for qp in qps.values():
    print(qp.model.solve())

solution for: three_vars_quadratic
objective: 5
x=1
z=4

solution for: testprob
objective: 54
X1=4
X2=-1
X3=6



# Solve with DWave Leap and CliqueSampler

In [6]:
solvers = [createSolver(LeapHybridSampler()), createSolver(DWaveCliqueSampler(),1000)]
results = {}
for qp_name in qps.keys():
    print(qp_name)
    

three_vars_quadratic
testprob


In [7]:
qp_name = "testprob"
results[qp_name] = [optimizer.solve(qps[qp_name]) for optimizer in solvers]
# problem ids : 
# Leap db6c6b80-8882-4636-9879-a18c3b57c980
# Advantage system 0b80397c-22a2-41e1-9dce-2c6cffca3ff3

In [8]:
for res in results[qp_name]:
    print(res)

optimal function value: 54.0
optimal value: [ 4. -1.  6.]
status: SUCCESS
optimal function value: 54.0
optimal value: [ 4. -1.  6.]
status: SUCCESS


## So we found the solution

## now test a quadratic constraint

In [10]:
qp_name = "three_vars_quadratic"
results[qp_name] = [optimizer.solve(qps[qp_name]) for optimizer in solvers]

QiskitOptimizationError: 'Incompatible problem: Quadratic constraints are not supported. '